Tools necessary for WGS variant calling using GATK include:

Picard is a Java-based toolkit for manipulating high-throughput sequencing (HTS) data and formats such as SAM/BAM/CRAM and VCF.

GATK is a toolkit for variant discovery in high-throughput sequencing data. It is designed to work best with data generated by next-generation DNA sequencing technologies, but can be applied to other types of data as well. 

Samtools is a suite of programs for interacting with high-throughput sequencing data. It provides various utilities for post-processing alignments in the SAM, BAM and CRAM formats, such as indexing, variant calling, viewing alignments, and generating alignments in a per-position format. It also includes a large number of utilities for manipulating alignments in a variety of different ways, for example, to extract all reads that map to a particular region of the genome.

BWA is a software package for mapping low-divergent sequences against a large reference genome, such as the human genome. It consists of three algorithms: BWA-backtrack, BWA-SW and BWA-MEM. The first algorithm is designed for Illumina sequence reads up to 100bp, while the rest two for longer sequences ranged from 70bp to 1Mbp. BWA-MEM and BWA-SW share similar features such as long-read support and split alignment, but BWA-MEM, which is the latest, is generally recommended for high-quality queries as it is faster and more accurate. BWA-MEM also has better performance than BWA-backtrack for 70-100bp Illumina reads.  For all the algorithms, BWA first needs to construct the FM-index for the reference genome (the index command). Alignment algorithms are invoked with different sub-commands: aln/samse/sampe for BWA-backtrack, bwasw for BWA-SW and mem for the BWA-MEM algorithm.

FastQC is a quality control tool for high throughput sequence data. It provides a modular set of analyses which you can use to give a quick impression of whether your data has any problems of which you should be aware before doing any further analysis. It produces a simple HTML file with a summary of the results of the analysis, which can be viewed in a web browser.

Other tools that are useful for WGS variant calling include:
- Freebayes, a Bayesian genetic variant detector designed to find small polymorphisms, specifically SNPs (single-nucleotide polymorphisms), indels (insertions and deletions), MNPs (multi-nucleotide polymorphisms), and complex events (composite insertion and substitution events)
- bcftools, a set of utilities that manipulate variant calls in the Variant Call Format (VCF) and its binary counterpart BCF
- vcftools, a set of tools written in Perl and C++ for working with VCF files, such as those generated by the 1000 Genomes Project
- bedtools, a suite of utilities for genome arithmetic
- tabix, a generic indexer for TAB-delimited genome position files
- bgzip, a small utility that enables tabix to work with gzip-compressed files

# Reference genome

download reference genome

In [1]:
REF_URL = "https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz"

In [2]:
import os

REF_DIR = "/mnt/nas/wgs/hg38"
REF_FILE = os.path.join(REF_DIR, "hg38.fa")

In [3]:
# Download the reference genome if the file doesn't exist
if not os.path.exists(os.path.join(REF_DIR, "hg38.fa")):
    !wget -O $REF_DIR/hg38.fa.gz $REF_URL
    !gunzip $REF_DIR/hg38.fa.gz

In [4]:
# index the reference genome file before running haplotype caller
if not os.path.exists(os.path.join(REF_DIR, "hg38.fa.fai")):
    !samtools faidx $REF_DIR/hg38.fa

In [5]:
# Create dictionary file for the reference genome if it doesn't exist
if not os.path.exists(os.path.join(REF_DIR, "hg38.dict")):
    !gatk CreateSequenceDictionary R=$REF_DIR/hg38.fa O=$REF_DIR/hg38.dict

In [6]:
SITE_URL = "https://storage.googleapis.com/genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.dbsnp138.vcf"
SITE_IDX_URL = "https://storage.googleapis.com/genomics-public-data/resources/broad/hg38/v0/Homo_sapiens_assembly38.dbsnp138.vcf.idx"

# Download known sites files for BQSR (Base Quality Score Recalibration) from GATK resource bundle if they don't exist
if not os.path.exists(os.path.join(REF_DIR, "Homo_sapiens_assembly38.dbsnp138.vcf")):
    !wget -O $REF_DIR $SITE_URL

if not os.path.exists(os.path.join(REF_DIR, "Homo_sapiens_assembly38.dbsnp138.vcf.idx")):
    !wget -O $REF_DIR $SITE_IDX_URL

In [ ]:
# index the reference genome file using BWA
!bwa index $REF_FILE

[bwa_index] Pack FASTA... 24.21 sec
[bwa_index] Construct BWT for the packed sequence...
[BWTIncCreate] textLength=6418572210, availableWord=463634060
[BWTIncConstructFromPacked] 10 iterations done. 99999986 characters processed.
[BWTIncConstructFromPacked] 20 iterations done. 199999986 characters processed.
[BWTIncConstructFromPacked] 30 iterations done. 299999986 characters processed.
[BWTIncConstructFromPacked] 40 iterations done. 399999986 characters processed.
[BWTIncConstructFromPacked] 50 iterations done. 499999986 characters processed.
[BWTIncConstructFromPacked] 60 iterations done. 599999986 characters processed.
[BWTIncConstructFromPacked] 70 iterations done. 699999986 characters processed.
[BWTIncConstructFromPacked] 80 iterations done. 799999986 characters processed.
[BWTIncConstructFromPacked] 90 iterations done. 899999986 characters processed.
[BWTIncConstructFromPacked] 100 iterations done. 999999986 characters processed.
[BWTIncConstructFromPacked] 110 iterations done. 

In [13]:
# show the contents of the reference directory
!ls -hal $REF_DIR

total 19G
drwxrwxr-x 2 acemap users 4.0K 1月   9 13:30 .
drwxrwxr-x 6 acemap users 4.0K 1月   9 10:16 ..
-rw-rw-r-- 1 acemap users  55K 1月   5 19:45 hg38.dict
-rw-rw-r-- 1 acemap users 3.1G 1月  16  2014 hg38.fa
-rw-rw-r-- 1 acemap users  21K 1月   9 13:14 hg38.fa.amb
-rw-rw-r-- 1 acemap users  22K 1月   9 13:14 hg38.fa.ann
-rw-rw-r-- 1 acemap users 3.0G 1月   9 13:14 hg38.fa.bwt
-rw-rw-r-- 1 acemap users  19K 1月   5 19:43 hg38.fa.fai
-rw-rw-r-- 1 acemap users 766M 1月   9 13:14 hg38.fa.pac
-rw-rw-r-- 1 acemap users 1.5G 1月   9 13:31 hg38.fa.sa
-rw-rw-r-- 1 acemap users  11G 7月  22  2016 Homo_sapiens_assembly38.dbsnp138.vcf
-rw-rw-r-- 1 acemap users  12M 7月  22  2016 Homo_sapiens_assembly38.dbsnp138.vcf.idx


# data

In [8]:
READS_DIR = "/mnt/nas/wgs/RDD2206P0001"

# Quality control

In [9]:
KNOWN_SITES = os.path.join(REF_DIR, "Homo_sapiens_assembly38.dbsnp138.vcf")
READS_FILE1 = os.path.join(READS_DIR, "RDD2206P0001-WGS-1_R1.fastq.gz")
READS_FILE2 = os.path.join(READS_DIR, "RDD2206P0001-WGS-1_R2.fastq.gz")


This is a quick overview of the quality of the data. We will use FastQC to generate a report for each sample.



In [11]:
# check the reads file quality using fastqc
!fastqc $READS_FILE1 $READS_FILE2 -o $READS_DIR

Started analysis of RDD2206P0001-WGS-1_R1.fastq.gz
Approx 5% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 10% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 15% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 20% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 25% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 30% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 35% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 40% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 45% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 50% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 55% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 60% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 65% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 70% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 75% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 80% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 85% complete for RDD2206P0001-WGS-1_R1.fastq.gz
Approx 90% comp

# Alignment

align reads to reference genome

The BWA-MEM algorithm performs local alignment. It may produce multiple primary alignments for different part of a query sequence. This is a crucial feature for long sequences. However, some tools such as Picard’s `markDuplicates` does not work with split alignments. One may consider to use option `-M` to flag shorter split hits as secondary. This option is not recommended for Illumina short reads. 

In [14]:
# align the reads to the reference genome using BWA
!bwa mem -t 50 -M -R "@RG\tID:RDD2206P0001\tSM:RDD2206P0001" $REF_FILE $READS_FILE1 $READS_FILE2 > $READS_DIR/RDD2206P0001-WGS-1.sam


[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 3333334 sequences (500000100 bp)...
[M::process] read 3333334 sequences (500000100 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (18, 1187807, 103, 33)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (169, 358, 951)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 2515)
[M::mem_pestat] mean and std.dev: (513.94, 472.41)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 3297)
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (330, 404, 475)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (40, 765)
[M::mem_pestat] mean and std.dev: (398.40, 114.16)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 910)
[M::mem_pestat] analyzing insert size distribution for orientation RF...
[M::mem_pestat] (25, 50, 

`-R` marks the complete read group header line. ’\t’ can be used in STR and will be converted to a TAB in the output SAM. The read group ID will be attached to every read in the output.


# Mark duplicates

mark the duplicates in the bam file

In [16]:
# mark duplicates in the aligned reads, using Spark
!gatk MarkDuplicatesSpark -I $READS_DIR/RDD2206P0001-WGS-1.sam -O $READS_DIR/RDD2206P0001-WGS-1.markdup.spark.bam -M $READS_DIR/RDD2206P0001-WGS-1.markdup.spark.metrics


Using GATK jar /home/ma/miniconda3/envs/insight/share/gatk4-4.3.0.0-0/gatk-package-4.3.0.0-local.jar
Running:
    java -Dsamjdk.use_async_io_read_samtools=false -Dsamjdk.use_async_io_write_samtools=true -Dsamjdk.use_async_io_write_tribble=false -Dsamjdk.compression_level=2 -jar /home/ma/miniconda3/envs/insight/share/gatk4-4.3.0.0-0/gatk-package-4.3.0.0-local.jar MarkDuplicatesSpark -I /mnt/nas/wgs/RDD2206P0001/RDD2206P0001-WGS-1.sam -O /mnt/nas/wgs/RDD2206P0001/RDD2206P0001-WGS-1.markdup.spark.bam -M /mnt/nas/wgs/RDD2206P0001/RDD2206P0001-WGS-1.markdup.spark.metrics
19:28:40.433 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/ma/miniconda3/envs/insight/share/gatk4-4.3.0.0-0/gatk-package-4.3.0.0-local.jar!/com/intel/gkl/native/libgkl_compression.so
19:28:40.580 INFO  MarkDuplicatesSpark - ------------------------------------------------------------
19:28:40.580 INFO  MarkDuplicatesSpark - The Genome Analysis Toolkit (GATK) v4.3.0.0
19:28:40.580 INFO  MarkDu

# Quality recalibration

recalibrate the quality of the reads

# Collect alignment metrics

# Collect insert size metrics

# Variant calling

# Variant filtering

filter the variants to remove false positives

# extract SNPs and indels

SNP (single nucleotide polymorphism) and indel (insertion/deletion) are the most common types of variants. However, there are other types of variants, such as structural variants, which are not covered in this tutorial.

since we are only interested in SNPs and indels, we will extract them from the vcf file